In [1]:
#in order to use PySpark, always start with the following code
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.2 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

In [4]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3EFW2STIYIY0I|          1|            2|          2|   N|                Y|
|R12NR0R5A9F7FT|          5|            0|          0|   N|                Y|
|R1LSH74R9XAP59|          2|            0|          1|   N|                Y|
|R1QXUNTF76K7L6|          2|            0|          0|   N|                Y|
|R2F7DR75PS8NKT|          5|            0|          0|   N|                Y|
|R2C1DJSCC8UFS6|          3|            0|          0|   N|                Y|
|R1AXGS1W4YFXMX|          1|            0|          2|   N|                Y|
|R1XU1B93402SYJ|          1|            1|          1|   N|                Y|
|R2U432NB3OPVR0|          5|            0|          0|   N|                Y|
|R3R6FIMIOQ5SP9|          5|            0|          0|   N|     

In [38]:
#Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
filtered_vine_df = vine_df.filter("total_votes>=20").select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26CC1STPSUHBS|          1|           27|         39|   N|                N|
| R7K83D59T9KWS|          1|           16|         26|   N|                N|
|R2ATG5BPVILHUM|          1|           16|         25|   N|                N|
|R1Q71TAM5ZOYTK|          3|           57|         61|   N|                N|
| RR8HYI7C2XKMU|          5|           25|         25|   N|                Y|
|R1VDCSJE4CJJIR|          1|           88|         97|   N|                Y|
|R2PZMSDKOO1Q6W|          1|           29|         43|   N|                Y|
|R39OOYN0DOLINK|          3|           91|        111|   N|                N|
|R1PLNHJZ8BKIBG|          1|           44|         46|   N|                Y|
|R14G4YQXL17REH|          1|            4|         28|   N|     

In [39]:
filtered_vine_df = filtered_vine_df.filter("helpful_votes/total_votes>=0.50").select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26CC1STPSUHBS|          1|           27|         39|   N|                N|
| R7K83D59T9KWS|          1|           16|         26|   N|                N|
|R2ATG5BPVILHUM|          1|           16|         25|   N|                N|
|R1Q71TAM5ZOYTK|          3|           57|         61|   N|                N|
| RR8HYI7C2XKMU|          5|           25|         25|   N|                Y|
|R1VDCSJE4CJJIR|          1|           88|         97|   N|                Y|
|R2PZMSDKOO1Q6W|          1|           29|         43|   N|                Y|
|R39OOYN0DOLINK|          3|           91|        111|   N|                N|
|R1PLNHJZ8BKIBG|          1|           44|         46|   N|                Y|
|R3UTVKJF44E0R6|          1|           46|         69|   N|     

In [44]:
paid_vine_df = filtered_vine_df.filter(filtered_vine_df.vine == "Y").select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28EIR3Z6RRUNN|          2|           50|         60|   Y|                N|
|R2YO2RESXZEWOF|          4|           24|         25|   Y|                N|
| R8GEE2EW3D6S9|          3|           49|         54|   Y|                N|
|R33W9V1MX6WKPS|          4|           47|         53|   Y|                N|
|R24E3W2ZRIFG8C|          5|           91|         93|   Y|                N|
|R2OR3KIFFG4YWT|          4|           75|         81|   Y|                Y|
|R1RLJAAFOA5MMS|          2|           25|         27|   Y|                N|
| RVYHD7IZ7W99B|          1|           78|         78|   Y|                N|
|R12EV95D2T56HA|          5|           37|         37|   Y|                N|
|R296NVZGV4VZLP|          5|           41|         44|   Y|     

In [43]:
unpaid_vine_df = filtered_vine_df.filter(filtered_vine_df.vine == "N").select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26CC1STPSUHBS|          1|           27|         39|   N|                N|
| R7K83D59T9KWS|          1|           16|         26|   N|                N|
|R2ATG5BPVILHUM|          1|           16|         25|   N|                N|
|R1Q71TAM5ZOYTK|          3|           57|         61|   N|                N|
| RR8HYI7C2XKMU|          5|           25|         25|   N|                Y|
|R1VDCSJE4CJJIR|          1|           88|         97|   N|                Y|
|R2PZMSDKOO1Q6W|          1|           29|         43|   N|                Y|
|R39OOYN0DOLINK|          3|           91|        111|   N|                N|
|R1PLNHJZ8BKIBG|          1|           44|         46|   N|                Y|
|R3UTVKJF44E0R6|          1|           46|         69|   N|     

In [55]:
num_paid = paid_vine_df.count()
five_star_paid = paid_vine_df.filter(paid_vine_df.star_rating == 5).select(["review_id", "star_rating"])
five_star_paid_num = five_star_paid.count()
paid_five_percent = five_star_paid_num/num_paid*100

print(f"There are {paid_vine_df.count()} number of paid reviews. Of those, {five_star_paid_num} are five stars which is {paid_five_percent}%.")

There are 248 number of paid reviews. Of those, 102 are five stars which is 41.12903225806452%.


In [57]:
num_unpaid = unpaid_vine_df.count()
five_star_unpaid = unpaid_vine_df.filter(unpaid_vine_df.star_rating == 5).select(["review_id", "star_rating"])
five_star_unpaid_num = five_star_unpaid.count()
unpaid_five_percent = five_star_unpaid_num/num_unpaid*100

print(f"There are {unpaid_vine_df.count()} number of paid reviews. Of those, {five_star_unpaid_num} are five stars which is {unpaid_five_percent}%.")

There are 17514 number of paid reviews. Of those, 5154 are five stars which is 29.427886262418635%.
